***
## Python shell

С интерпретатором кода Python, как и со многими другими программами, можно работать через командную строку. Если в консоли выполнить команду `python`, то интерпретатор Python запустится в «интерактивном режиме». После этого в командную строку можно вводить любые инструкции на Python — и они будут выполняться прямо в терминале.

Это похоже на работу командной строки, но вместо команд для работы с файлами выполняется программный код, строчка за строчкой. 

Откройте новое окно терминала и запустите **Python shell**. Символы `>>>` — это приглашение для ввода команд, то же, что и знак `$` в командной строке.

```bash
# Запускаем интерпретатор:
$ python
# Теперь можно писать код на python:
# Создаём переменную:
>>> best_slogan = 'Ложки нет! '
# Значение переменной можно вывести без функции print():
>>> best_slogan
# Получаем результат:
Ложки нет! 
# Арифметика тоже работает:
>>> multiplicator = 2 * 2
>>> multiplicator
4
# Python shell ничего не забывает (пока консоль не закрыта).
# Значения переменных хранятся в памяти:
>>> best_slogan * multiplicator
Ложки нет! Ложки нет! Ложки нет! Ложки нет!
```

Переменные, созданные в Python shell, будут доступны до тех пор, пока открыто окно терминала.

***
## Django shell

В Django есть множество полезных инструментов для разработки, и один из них — Django shell, инструмент для управления проектом из консоли. Это Python shell, предназначенный для работы с Django-проектом.

В Django shell можно импортировать любые сущности проекта — views, urls, model, любые другие пакеты или модули и выполнять в консоли любые операции, доступные в Django.

Активируйте виртуальное окружение проекта «Анфиса для друзей»; сам Django-проект запускать не нужно. Перейдите в директорию, где содержится файл manage.py и выполните команду запуска Django shell:

```bash
python manage.py shell 
```

В зависимости от вашей операционной системы и версии Python ответ в консоли может немного отличаться; выглядеть он будет примерно так:

```bash
Python 3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)] on win32
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
>>>
```

***
## Операции CRUD и работа с записями в БД через ORM

Операции с реляционными базами данных делят на четыре группы:

* **Create** — создание записи

* **Read (Retrieve)** — чтение записей

* **Update** — изменение записи

* **Delete** — удаление записи

Этот набор операций принято называть по первым буквам перечисленных групп: **CRUD-операции**. Пройдёмся по всем и посмотрим, как CRUD реализуется через Django ORM.

В классе `models.Model` есть специальный интерфейс `objects`; с помощью его методов и выполняются запросы к БД.

Начнём с метода `.create()` и создадим запись в таблице.

В базе данных проекта «Анфиса для друзей», который развёрнут на вашем компьютере, есть несколько таблиц. Проводить эксперименты проще на таблице, данные в которой не зависят от других таблиц, так что возьмём `ice_cream_category`.

***
## Создаём запись (Create)

За эту таблицу отвечает модель `Category` из *ice_cream/models.py*. 

```python
class Category(PublishedModel):
    title = models.CharField(max_length=256)
    slug = models.SlugField(max_length=64, unique=True)
    output_order = models.PositiveSmallIntegerField(default=100)
    ... 
```

* `id`: автоинкрементный первичный ключ типа `BigAutoField` (создан автоматически);

* `is_published`: тип `Bool`, значение по умолчанию — `True` (унаследован от абстрактной модели `PublishedModel`);

* `title`: тип `CharField`, максимальная длина строки — 256 символов;

* `slug`: тип `SlugField`, максимальная длина строки — 64 символа;

* `output_order`: тип `PositiveSmallIntegerField`, значение по умолчанию — 100.

Создайте новую запись в таблице `ice_cream_category` в базе данных с помощью метода `.create()`:

```py
# Импортируем модель, с которой планируем работать.
>>> from ice_cream.models import Category

# Cоздаём объект, передаём значения атрибутов. 
# Поля со значениями по умолчанию не заполняем: пусть это сделает Django ORM.
# В поле id тоже не передаём значение.
>>> Category.objects.create(title='Категория, созданная через shell', slug='shell_category')

# По нажатию Enter этот код выполнится, и на основе значений, 
# переданных в метод create(), будет создана новая запись в БД.
# О чём и будет сообщено вот такой строкой.
<Category: Category object (1)>
```

При вызове метода .create() будет отправлен такой запрос к базе данных:

```sql
INSERT INTO ice_cream_category(is_published, title, slug, output_order)
VALUES (1, 'Категория, созданная через shell', 'shell_category', 100); 
```

Создадим ещё одну запись в БД:

```py
# Обращаемся к интерфейсу objects модели Category и вызываем метод .create():
>>> Category.objects.create(is_published=False, title='Ещё одна категория, созданная через shell', slug='one_more_shell_category', output_order=90)
# Получаем подтверждение:
<Category: Category object (2)> 
```

***
## Получение информации (Read или Retrieve)

В БД сейчас сохранены две записи. Извлечём их.

При получении записей из БД каждая запись преобразуется в объект модели; Django ORM возвращает специальный объект **QuerySet**, он содержит список объектов модели, соответствующих условиям запроса. 

Запросим все объекты модели `Category` — для этого применяется метод `.all()`:

```py
# Модели уже импортированы (если вы не закрывали консоль), 
# повторно их импортировать не надо: Shell всё помнит!
>>> categories = Category.objects.all()

# Что в переменной categories?
>>> categories

# Django shell сообщает, что QuerySet содержит два объекта модели Category с pk 1 и 2:
<QuerySet [<Category: Category object (1)>, <Category: Category object (2)>]>
```

Одна из самых востребованных задач при работе с базой — поиск объектов по заданным признакам. В SQL за это отвечают команды блока `WHERE`, а в Django ORM — метод `.filter()`:

```py
# Найти все объекты, значение поля is_published у которых равно True: 
# Получившийся результат можно не присваивать переменной, 
# а сразу вывести в консоль: 
>>> Category.objects.filter(is_published=True)
<QuerySet [<Category: Category object (1)>]>
# Вернулся QuerySet, в котором хранится один объект модели Category (с pk=1)
```


Чтобы получить отдельный объект, а не QuerySet, применяют метод `.get()`. 

Django ORM по умолчанию создаёт во всех таблицах поле с первичным ключом, и по этому ключу можно получить из БД нужную запись:

```py
>>> category = Category.objects.get(pk=1)
>>> category
# Получаем <Объект> модели Category с указанным в запросе pk:
<Category: Category object (1)>
```

> Поле с первичным ключом в таблице называется `id`, но в Django ORM к первичному ключу любой таблицы можно обращаться и по имени поля, и по псевдониму `pk`.

Значения полей записи преобразовались в атрибуты объекта, и к ним можно обратиться стандартным в Python способом, через точечную нотацию: 

```py
# Проверим, какой id присвоен новому объекту в базе:
>>> category.id  # Нажимаем Enter, и Shell возвращает значение:
1

# Или через псевдоним
>>> category.pk
1

# А что в поле title?
>>> category.title
'Категория, созданная через shell'

# А в поле slug?
>>> category.slug
'shell_category'

# А вот что в полях is_published и output_order:
>>> category.is_published
True
>>> category.output_order
100
```

Если метод `.get()` запрашивает несуществующий объект, то появится сообщение об ошибке:

```py
# Запросим несчастливый объект:
>>> Category.objects.get(pk=13)
Traceback (most recent call last):
# ... тут много разной информации, где именно возникла ошибка ... 
raise self.model.DoesNotExist(
ice_cream.models.Category.DoesNotExist: Category matching query does not exist. 
```

***
## Обновление информации (Update)

Изменить объект можно двумя способами.

Можно получить коллекцию объектов (запросить все объекты или отфильтровать их по какому-то параметру) — и присвоить одному или нескольким полям этих объектов новые значения:

```py
# Category.objects.all() возвращает QuerySet со всеми объектами модели,
# а метод update() меняет свойства всех объектов:
>>> Category.objects.all().update(title='Изменённое поле категории', is_published=True)
# В ответ получим количество изменённых записей
2 
```

Этот способ подойдёт даже в том случае, если QuerySet содержит только один объект.

А можно пойти другим путём: получить объект, присвоить новое значение одному из его полей и вызвать метод `.save()`:

```py
# Получаем объект и сохраняем его в переменную category_for_change:
>>> category_for_change = Category.objects.get(pk=1)
# Меняем значение одного из полей: 
>>> category_for_change.title = 'Ещё раз изменённое поле категории'
>>> category_for_change.is_published = False
# Новое значение присвоено объекту модели, но в БД всё ещё хранится старое значение.
# Чтобы отправить новое значение в базу данных — вызываем метод save():
>>> category_for_change.save()

# Смотрим информацию из обновлённых полей:
>>> Category.objects.get(pk=1).title 
'Ещё раз изменённое поле категории'
>>> Category.objects.get(pk=1).is_published
False
# Ура, получилось!
```

Этот способ выглядит сложнее, но при работе с одиночным объектом приходится использовать именно его. Метод `.update()` можно применить для QuerySet, но не для отдельного объекта модели.

***
## Удаление записи (Delete)

Для удаления объектов применяют метод `.delete()`. 

Объекты можно удалять поштучно: получить нужный объект из базы и вызвать для него метод `.delete()`. 

Удалим все созданные в этом уроке записи в БД:

```py
>>> category_for_delete = Category.objects.get(pk=1)
>>> category_for_delete.delete()
# Будет выведено
(1, {'ice_cream.Category': 1})
>>> category_for_delete = Category.objects.get(pk=2)
>>> category_for_delete.delete()
# Будет выведено
(1, {'ice_cream.Category': 1})

# Проверяем, что объекты действительно удалились:
>>> Category.objects.all()
<QuerySet []>
# Совсем пусто: нет ни одного объекта класса Category.  
```

А можно удалить все объекты оптом:

```py
# Получаем QuerySet и удаляем все содержащиеся в нём объекты:
>>> Category.objects.all().delete() 
```